In [2]:
import numpy as np
import matplotlib.pyplot as plt
import cmath

In [24]:
#import data generated in MATLAB
physical_real=np.loadtxt(r'/Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Physical_matrices_Dim=4_RealPart.txt')
Non_physical_real=np.loadtxt(r'//Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Non_Physical_matrices_Dim=4_RealPart.txt')
physical_imag=np.loadtxt(r'//Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Physical_matrices_Dim=4_ImagPart.txt')
Non_physical_imag=np.loadtxt(r'//Users/noemitagliavacche/Documents/PhD/COURSES/MACHINE_LEARNING/Exam/MATLAB_dataset_creation/300_Non_Physical_matrices_Dim=4_ImagPart.txt')

################ TO CREATE THE DATASET #############

#Classifier: 0-> Physical matrix, 1-> Non physical matrix
target_Physical=np.zeros((physical_real.shape[0],1),dtype=int)
target_NonPhysical=np.ones((physical_real.shape[0],1),dtype=int)

Physical_real=np.concatenate((physical_real,target_Physical), axis=1)
Physical_imag=np.concatenate((physical_imag,target_Physical), axis=1)
NonPhysical_real=np.concatenate((Non_physical_real,target_NonPhysical), axis=1)
NonPhysical_imag=np.concatenate((Non_physical_imag,target_NonPhysical), axis=1)
Physical_data=np.stack((Physical_real,Physical_imag),axis=2)
NonPhysical_data=np.stack((NonPhysical_real,NonPhysical_imag),axis=2)
#print("Physical data size:  ", Physical_data.shape, ", Non Physical data size: ", NonPhysical_data.shape)

dataset=np.concatenate((Physical_data, NonPhysical_data), axis=0)
print("Dataset dimension", dataset.shape)

features=dataset[:,:-1,:]
target=dataset[:,-1,1]
print("features size: ", features.shape, ", target size: ", target.shape)


Dataset dimension (600, 17, 2)
features size:  (600, 16, 2) , target size:  (600,)


In [25]:
from sklearn.model_selection import train_test_split
X_train, X_val, Y_train, Y_val = train_test_split(features, target, test_size=0.33, shuffle=True)
print("X_train: ",X_train.shape,", X_val: " ,X_val.shape, ", Y_train: ", Y_train.shape, ", Y_val: ",Y_val.shape)

X_train:  (402, 16, 2) , X_val:  (198, 16, 2) , Y_train:  (402,) , Y_val:  (198,)


In [ ]:
# keras imports
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Conv1D, AveragePooling1D, Dropout, Flatten
from tensorflow.keras.utils import plot_model
from tensorflow.keras import backend as K
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, TerminateOnNaN

In [ ]:
featureArrayLength = (X_train.shape[1],X_train.shape[2])
dropoutRate = 0.25

In [ ]:
####
inputList = Input(shape=(featureArrayLength))
x = Conv1D(20, kernel_size=3, data_format="channels_last", strides=1, padding="valid", activation='relu')(inputList)
x = AveragePooling1D(pool_size=3)(x)
#
x = Conv1D(40, kernel_size=3, data_format="channels_last", strides=1, padding="valid", activation='relu')(x)
x = AveragePooling1D(pool_size=3)(x)
#
x = Conv1D(60, kernel_size=2, data_format="channels_last", strides=1, padding="valid", activation='relu')(x)
x = AveragePooling1D(pool_size=9)(x)
#
x = Flatten()(x)
x = Dense(20, activation='relu')(x)
x = Dropout(dropoutRate)(x)
x = Dense(10, activation='relu')(x)
x = Dropout(dropoutRate)(x)
output = Dense(5, activation='softmax')(x)
####
model = Model(inputs=inputList, outputs=output)

In [ ]:
model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

In [ ]:
batch_size = 128
n_epochs = 200

# train 
history = model.fit(X_train, Y_train, epochs=n_epochs, batch_size=batch_size, verbose = 2,
                validation_data=(X_val, Y_val),
                callbacks = [
                EarlyStopping(monitor='val_loss', patience=10, verbose=1),
                ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=2, verbose=1),
                TerminateOnNaN()])

In [ ]:
# plot training history
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.yscale('log')
plt.title('Training History')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['training', 'validation'], loc='upper right')
plt.show()